# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/output/cities.csv")
cities_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,jamestown,42.0970,-79.2353,63.99,76,10,1.99,US,1666651518
1,saint-philippe,-21.3585,55.7679,69.39,82,36,6.55,RE,1666651657
2,te anau,-45.4167,167.7167,50.59,51,11,2.37,NZ,1666651635
3,sao joao da barra,-21.6403,-41.0511,75.29,87,56,6.89,BR,1666651658
4,ilulissat,69.2167,-51.1000,25.84,87,100,4.83,GL,1666651658
...,...,...,...,...,...,...,...,...,...
559,qom,34.6401,50.8764,65.70,43,84,0.60,IR,1666651974
560,daru,-9.0763,143.2092,84.69,68,26,8.30,PG,1666651974
561,ewa beach,21.3156,-158.0072,89.55,53,40,19.57,US,1666651631
562,bowen,-20.0167,148.2333,82.24,77,44,16.42,AU,1666651975


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [10]:

locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

* A max temperature lower than 80 degrees but higher than 70.

* Wind speed less than 10 mph.

* Zero cloudiness.

* Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

* Note: Feel free to adjust your specifications, but make sure to limit the number of rows returned by your API requests to a reasonable number.

In [11]:
new_df = cities_df[(cities_df["Max Temp"] >= 70)&
                       (cities_df["Max Temp"] <= 80)&
                       (cities_df["Wind Speed"] <= 10)&
                       (cities_df["Cloudiness"] == 0)].dropna()
new_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
112,marrakesh,31.6315,-8.0083,75.27,27,0,1.14,MA,1666651716
124,guerrero negro,27.9769,-114.0611,75.99,38,0,7.27,MX,1666651723
165,sturgis,41.7992,-85.4192,70.05,51,0,8.05,US,1666651745
169,saint-pierre,-21.3393,55.4781,70.92,68,0,5.75,RE,1666651748
194,mormugao,15.4000,73.8000,70.86,94,0,4.61,IN,1666651762
229,umm lajj,25.0213,37.2685,78.31,53,0,8.41,SA,1666651781
260,semnan,35.5729,53.3971,71.44,28,0,9.22,IR,1666651797
384,vila velha,-20.3297,-40.2925,73.35,89,0,3.44,BR,1666651714
397,montes claros,-16.7350,-43.8617,74.98,73,0,1.14,BR,1666651880
425,bay city,43.5945,-83.8889,73.62,43,0,9.22,US,1666651896


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = new_df
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
112,marrakesh,31.6315,-8.0083,75.27,27,0,1.14,MA,1666651716,
124,guerrero negro,27.9769,-114.0611,75.99,38,0,7.27,MX,1666651723,
165,sturgis,41.7992,-85.4192,70.05,51,0,8.05,US,1666651745,
169,saint-pierre,-21.3393,55.4781,70.92,68,0,5.75,RE,1666651748,
194,mormugao,15.4000,73.8000,70.86,94,0,4.61,IN,1666651762,
229,umm lajj,25.0213,37.2685,78.31,53,0,8.41,SA,1666651781,
260,semnan,35.5729,53.3971,71.44,28,0,9.22,IR,1666651797,
384,vila velha,-20.3297,-40.2925,73.35,89,0,3.44,BR,1666651714,
397,montes claros,-16.7350,-43.8617,74.98,73,0,1.14,BR,1666651880,
425,bay city,43.5945,-83.8889,73.62,43,0,9.22,US,1666651896,


In [14]:
#Create URL and loop to gather information
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# set up a parameters dictionary
params = {
    "type": "hotel",
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key}

In [17]:
# geocoordinates
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    #change params for location
    params["location"] = f"{lat},{lng}"
    
    response = requests.get(base_url, params=params).json()
    
    #to save hotel information
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        #hotel_df.append(response['results'][0]['name'])
    except IndexError:
        print("Hotels not found")

In [18]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
112,marrakesh,31.6315,-8.0083,75.27,27,0,1.14,MA,1666651716,Mamounia Hotel
124,guerrero negro,27.9769,-114.0611,75.99,38,0,7.27,MX,1666651723,The Halfway Inn
165,sturgis,41.7992,-85.4192,70.05,51,0,8.05,US,1666651745,Hampton Inn Sturgis-Lagrange Area
169,saint-pierre,-21.3393,55.4781,70.92,68,0,5.75,RE,1666651748,Lindsey Hôtel
194,mormugao,15.4000,73.8000,70.86,94,0,4.61,IN,1666651762,Hotel Annapurna
229,umm lajj,25.0213,37.2685,78.31,53,0,8.41,SA,1666651781,Juman Hotel
260,semnan,35.5729,53.3971,71.44,28,0,9.22,IR,1666651797,Semnan Tourist Hotel
384,vila velha,-20.3297,-40.2925,73.35,89,0,3.44,BR,1666651714,Hotel Golden Tulip Porto Vitória
397,montes claros,-16.7350,-43.8617,74.98,73,0,1.14,BR,1666651880,ibis Montes Claros Shopping
425,bay city,43.5945,-83.8889,73.62,43,0,9.22,US,1666651896,DoubleTree by Hilton Hotel Bay City - Riverfront


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))